In [1]:
from modules import process as prc
import pandas
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

In [2]:
train_data = pandas.read_csv('../data/training_data.csv', sep=',', header=0)
test_data = pandas.read_csv('../data/testing_data.csv', sep=',', header=0)

X_train = train_data.iloc[:, 0:5]
X_test = test_data.iloc[:, 0:5]

y_train = train_data.iloc[:, -1]
y_test = test_data.iloc[:, -1]

FileNotFoundError: [Errno 2] File b'../data/test.csv' does not exist: b'../data/test.csv'

In [ ]:
# bernoulli
BernNP = BernoulliNB(binarize = True)
BernNP.fit(X_train, y_train)

y_pred = BernNP.predict(X_test)
print(prc.accuracy_score(y_pred, y_test))

In [ ]:
# Gaussian
gaussNB = GaussianNB()
gaussNB.fit(X_train, y_train)

y_pred = gaussNB.predict(X_test)

print(prc.accuracy_score(y_pred, y_test))